# 盘中动态:20241122

In [1]:
import nbimporter
from 盘前作业20241122 import plotStock

repday='20241122'

## 1. 今日板块热点


#### 2.2.1 集合竞价概念板块排名

In [2]:
import datetime
import os
import pandas as pd

#记录排名
# basename: 板块类型（概念板块、行业板块）
# start_time, end_time: 排名时间段
# sector: 板块名称
def recordRank(basename,start_time,end_time):
    now = datetime.datetime.now()
    tStr = end_time.strftime("%Y%m%d%H%M")
    csvfile=f'{basename}{tStr}.txt'
    if now.time() < end_time and now.time() >= start_time:
        if basename=='conceptRank':
            rank=ak.stock_board_concept_name_em()
        elif basename=='industryRank':
            rank=ak.stock_board_industry_name_em()
        rank.to_csv(csvfile)
    else:
        if os.path.exists(csvfile):
            rank=pd.read_csv(csvfile)
        else:
            print('非指定时间段，没有数据源.')
            rank=pd.DataFrame()
    if not rank.empty:
        mvtotal=rank['总市值'].sum()
        rank['市值比']=rank['总市值']/mvtotal
        rank['上涨家数%']=rank['上涨家数']/(rank['上涨家数']+rank['下跌家数'])
        columns=['排名','板块名称','板块代码','涨跌幅','市值比','换手率','上涨家数%','领涨股票','领涨股票-涨跌幅']
        rank=rank[columns]
    return rank

#排名靠前的板块的股票明细排名
# sectorRankDf: 板块排名DataFrame(recordRank生成)
# sectorNum,stockNum: 返回板块数、股票数
def top5sectorStocks(basename,start_time,end_time,sectorRankDf,sectorNum=5,stockNum=5):
    top5={}
    now = datetime.datetime.now()
    tStr = start_time.strftime("%Y%m%d%H%M")
    if sectorRankDf.empty:
        print('非指定时间段，没有数据源。')
    else:
        print('排名前5领涨板块成分股列表。')
        for sym in sectorRankDf['板块名称'].to_list()[:5]:
            csvfile=f'{basename}_{sym}{tStr}.txt'
            if now.time() < end_time and now.time() >= start_time:
                if basename=='conceptRank':
                    top5[sym]=ak.stock_board_concept_cons_em(sym)
                else:
                    top5[sym]=ak.stock_board_industry_cons_em(sym)
                top5[sym].to_csv(csvfile)
            else:
                if os.path.exists(csvfile):
                    top5[sym]=pd.read_csv(csvfile)
                else:
                    print('No record stored and not available from source.')
                    top5[sym]=pd.DataFrame()

            if not top5[sym].empty:
                atotal=top5[sym]['成交额'].sum()
                top5[sym]['成交额比']=top5[sym]['成交额']/atotal
                columns=['代码','名称','涨跌幅','成交额','换手率','市盈率-动态','市净率']
                top5[sym]=top5[sym][columns][:stockNum]
                print(f'板块：{sym}:\n{top5[sym]}\n\n')
    return top5   

In [3]:
#当天集合竞价阶段概念板块排名
basename='conceptRank'
start_time = datetime.time(9, 15)
end_time = datetime.time(9, 31)
rank0=recordRank(basename,start_time,end_time)
rank0.head(20)

,排名,板块名称,板块代码,涨跌幅,市值比,换手率,上涨家数%,领涨股票,领涨股票-涨跌幅
0,1,昨日连板_含一字,BK1051,3.15,0.000219,1.71,0.739130,魅视科技,9.99
1,2,可控核聚变,BK1163,3.10,0.000245,0.12,0.823529,永鼎股份,10.00
2,3,昨日涨停_含一字,BK1050,3.10,0.000585,0.52,0.793103,魅视科技,9.99
3,4,昨日涨停,BK0815,2.10,0.000508,0.40,0.762887,南极电商,10.03
4,5,超导概念,BK0679,2.03,0.000150,0.13,0.666667,永鼎股份,10.00
5,6,同步磁阻电机,BK1131,1.07,0.000029,0.03,0.400000,佳电股份,6.68
6,7,轮毂电机,BK1099,0.90,0.000035,0.02,0.666667,佳电股份,6.68
7,8,昨日连板,BK0816,0.68,0.000133,1.42,0.642857,南极电商,10.03
8,9,快递概念,BK0990,0.59,0.000382,0.12,0.631579,佛塑科技,7.73
9,10,湖北自贸,BK0926,0.48,0.000218,0.03,0.423077,奥美医疗,10.04


In [4]:
top5_concept_stocks=top5sectorStocks(basename,start_time,end_time,rank0)

排名前5领涨板块成分股列表。
板块：昨日连板_含一字:
       代码    名称    涨跌幅         成交额   换手率  市盈率-动态   市净率
0  600889  南京化纤  10.05   1088520.0  0.03  -19.49  5.78
1    3036  泰坦股份  10.04   7435428.0  0.28   24.79  1.97
2    2761  浙江建投  10.03  17281701.8  0.13   73.46  2.09
3    2127  南极电商  10.03  96665940.0  1.13  145.32  2.35
4    2611  东方精工  10.02  29970772.8  0.21   39.23  3.57


板块：可控核聚变:
       代码    名称    涨跌幅         成交额   换手率  市盈率-动态   市净率
0  430300  辰光医疗  13.29  18900112.0  1.43  -89.23  6.92
1  600105  永鼎股份  10.00  66214709.0  0.78  111.79  2.93
2  603015  弘讯科技   9.98  16222352.0  0.34   71.62  3.51
3    2255  海陆重工   6.71  72504052.0  1.70   17.28  1.38
4     969  安泰科技   6.64  96825300.0  0.73   30.13  2.49


板块：昨日涨停_含一字:
       代码    名称    涨跌幅         成交额   换手率  市盈率-动态   市净率
0  832145  恒合股份  14.02  16614355.0  2.18  -94.56  4.49
1  430300  辰光医疗  13.29  18900112.0  1.43  -89.23  6.92
2  600889  南京化纤  10.05   1088520.0  0.03  -19.49  5.78
3    3036  泰坦股份  10.04   7435428.0  0.28   24.79  1.97
4    2950 

#### 2.2.3 开盘后15分钟概念板块排名

In [5]:
start_time = datetime.time(9, 31)
end_time = datetime.time(9, 52)
rank1=recordRank(basename,start_time,end_time)
rank1.head(20)

非指定时间段，没有数据源.


""


排名前5概念板块领涨成分股

In [6]:
top5_concept_stocks=top5sectorStocks(basename,start_time,end_time,rank1)

非指定时间段，没有数据源。


#### 2.2.3 集合竞价行业板块涨跌幅

In [7]:
#当天集合竞价到开盘后30分钟涨跌幅排名
basename='industryRank'
start_time = datetime.time(9, 15)
end_time = datetime.time(9, 31)
rank3=recordRank(basename,start_time,end_time)
rank3.head(20)

,排名,板块名称,板块代码,涨跌幅,市值比,换手率,上涨家数%,领涨股票,领涨股票-涨跌幅
0,1,贵金属,BK0732,0.90,0.007726,0.06,0.818182,湖南黄金,9.98
1,2,电源设备,BK1034,0.72,0.003960,0.20,0.480000,融发核电,9.95
2,3,家用轻工,BK0440,0.50,0.002688,0.06,0.600000,实丰文化,10.00
3,4,贸易行业,BK0484,0.44,0.003321,0.13,0.593750,南极电商,10.03
4,5,珠宝首饰,BK0734,0.39,0.001412,0.01,0.466667,莱绅通灵,4.18
5,6,石油行业,BK0464,0.39,0.036949,0.00,0.571429,统一股份,10.03
6,7,物流行业,BK0422,0.28,0.008494,0.01,0.490196,华光源海,11.11
7,8,电子化学品,BK1039,0.18,0.002874,0.04,0.416667,光华科技,9.98
8,9,美容护理,BK1035,0.15,0.003008,0.05,0.478261,可靠股份,1.61
9,10,医疗器械,BK1041,0.11,0.013830,0.02,0.373626,奥美医疗,10.04


行业板块中成分股

In [8]:
top5_industry_stocks=top5sectorStocks(basename,start_time,end_time,rank3)

排名前5领涨板块成分股列表。
板块：贵金属:
       代码    名称   涨跌幅          成交额   换手率  市盈率-动态   市净率
0    2155  湖南黄金  9.98  165412872.0  0.75   24.88  3.29
1    2716  湖南白银  1.95   22158372.0  0.27   62.78  3.18
2  300139  晓程科技  1.81   13645380.0  0.31   93.78  5.15
3  600489  中金黄金  0.68   14614040.0  0.02   18.29  2.42
4     975  山金国际  0.64    1886382.0  0.00   20.94  3.86


板块：电源设备:
       代码    名称   涨跌幅           成交额   换手率  市盈率-动态   市净率
0    2366  融发核电  9.95  1.551016e+08  2.03 -726.18  2.73
1  600405   动力源  9.94  5.423415e+07  1.81  -15.88  5.35
2    2255  海陆重工  6.71  7.250405e+07  1.70   17.28  1.38
3  601727  上海电气  2.18  6.366842e+07  0.05  158.71  3.00
4  600875  东方电气  1.66  6.510353e+06  0.02   14.76  1.34


板块：家用轻工:
       代码    名称    涨跌幅         成交额   换手率  市盈率-动态   市净率
0    2862  实丰文化  10.00  8446810.00  0.31  593.01  8.66
1    1300   三柏硕   3.95  5975940.00  0.66  121.12  3.16
2  831445  龙竹科技   3.28  1351535.25  0.13   59.97  4.26
3  300729  乐歌股份   2.76  4184600.00  0.08   16.54  1.66
4  873132  泰鹏智

#### 2.2.4 开盘15分钟行业板块涨跌幅

In [9]:
basename='industryRank'
start_time = datetime.time(9, 31)
end_time = datetime.time(9, 51)
rank4=recordRank(basename,start_time,end_time)
rank4.head(20)

非指定时间段，没有数据源.


""


行业板块中成分股

### 2.2.5 集合竞价阶段涨停个股热点明细

In [10]:
#在指定时段获取涨停数据，若时间不在指定时段，则读取历史最后一次下载的记录。
def stockLimitUp(start_time,end_time):
    now = datetime.datetime.now()
    tStr = end_time.strftime("%Y%m%d%H%M")
    csvfile=f'limitUp{tStr}.txt'
    if now.time() < end_time and now.time() >= start_time:
        rank=ak.stock_zh_a_spot_em()
        rank.to_csv(csvfile)
    else:
        if os.path.exists(csvfile):
            rank=pd.read_csv(csvfile)
        else:
            print('No record stored and not available from source.')
            rank=pd.DataFrame()
    if not rank.empty:
        columns=['代码','名称','涨跌幅','量比','换手率','市盈率-动态','市净率','流通市值','涨速','5分钟涨跌','60日涨跌幅','年初至今涨跌幅']
        rank=rank[columns].sort_values(by='涨跌幅', ascending=False)
    return rank

In [11]:
start_time = datetime.time(9, 15)
end_time = datetime.time(9, 31)
openA = stockLimitUp(start_time,end_time)
openA.head(50)

,代码,名称,涨跌幅,量比,换手率,市盈率-动态,市净率,流通市值,涨速,5分钟涨跌,60日涨跌幅,年初至今涨跌幅
0,301631,N壹连,180.86,NaN,4.37,54.66,5.84,2.670802e+09,0.0,2.50,180.86,180.86
1,832145,恒合股份,14.02,62.07,2.18,-94.56,4.49,7.620632e+08,0.0,-12.28,253.43,117.39
2,430300,辰光医疗,13.29,24.82,1.43,-89.23,6.92,1.322050e+09,0.0,-4.35,214.29,43.04
3,872351,华光源海,11.11,20.30,1.62,157.87,3.00,5.289066e+08,0.0,0.00,162.70,64.84
4,301037,保立佳,11.07,22.33,0.58,-19.21,2.48,6.357264e+08,0.0,-1.92,44.44,-10.53
5,600889,南京化纤,10.05,1.85,0.03,-19.49,5.78,4.132383e+09,0.0,0.00,126.96,95.83
6,3036,泰坦股份,10.04,36.22,0.28,24.79,1.97,2.674163e+09,0.0,0.00,69.13,3.34
7,2950,奥美医疗,10.04,81.44,0.62,18.84,2.00,4.754990e+09,0.0,0.00,60.61,11.32
10,2127,南极电商,10.03,69.98,1.13,145.32,2.35,8.537334e+09,0.0,0.00,62.74,12.34
9,2761,浙江建投,10.03,13.19,0.13,73.46,2.09,1.364654e+10,0.0,0.00,74.07,23.60


### 2.2.6 开盘15分钟涨停个股热点明细


In [12]:
start_time = datetime.time(9, 31)
end_time = datetime.time(9, 53)
open15 = stockLimitUp(start_time,end_time)
open15.head(50)

No record stored and not available from source.


""


#### 2.2.7 板块动态评析

本轮行情启动股票热点追踪（需要从历史行情中组合分析，现有股票涨停池昨日和今日都不稳定）

In [13]:
from datetime import datetime, timedelta

def get_stock_zt_pool(start_date, end_date):
    current_date = datetime.strptime(start_date, '%Y%m%d')
    end_date = datetime.strptime(end_date, '%Y%m%d')
    
    concat_zt_df = pd.DataFrame()

    while current_date <= end_date:
        date_str = current_date.strftime('%Y%m%d')
        try:
            stock_zt_pool_strong_em_df = ak.stock_zt_pool_previous_em(date=date_str)
            stock_zt_pool_strong_em_df['日期'] = pd.to_datetime(date_str)
            concat_zt_df = pd.concat([concat_zt_df, stock_zt_pool_strong_em_df], ignore_index=True)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
        
        current_date += timedelta(days=1)
    
    return concat_zt_df

In [14]:
syclestart='20240925'
#ak.stock_zt_pool_strong_em(date=syclestart)

In [15]:
#本轮行情启动日
syclestart='20240924'
result_df = get_stock_zt_pool(syclestart, lastday)
result_df

NameError: name 'lastday' is not defined

## 4.个股市场


### 4.1 跟进股票列表
#### 4.1.1 历史K线

In [ ]:
def plotDcStockSingle(df,days):
    OCHLVDict={'日期':'Date','开盘':'Open','收盘':'Close','最高':'High','最低':'Low','成交量':'Volume'}
    print(f'{days}K线图')
    plotStock(df.tail(days),OCHLVDict)

In [ ]:
#根据代码获取股票信息
#symbol:若是字符串，则返回单个股信息；若是列表，则遍历元素，返回相应列表。
def getStockInfo(symbol):
    if type(symbol).__name__=='str':
        stocknamedf=ak.stock_individual_info_em(symbol=symbol)
        stockname=stocknamedf.loc[stocknamedf['item']=='股票简称','value'][1]
        stockIPO=stocknamedf.loc[stocknamedf['item']=='上市时间','value'][7]
        stockIndustry=stocknamedf.loc[stocknamedf['item']=='行业','value'][6]
        stockTAmount=stocknamedf.loc[stocknamedf['item']=='总股本','value'][2]
        stockFAmount=stocknamedf.loc[stocknamedf['item']=='流通股','value'][3]
        stockTValue=stocknamedf.loc[stocknamedf['item']=='总市值','value'][4]
        stockFValue=stocknamedf.loc[stocknamedf['item']=='流通市值','value'][5]
    else:
        stockname,stockIPO,stockIndustry,stockTAmount,stockFAmount,stockTValue,stockFValue=[],[],[],[],[],[],[]
        for sym in symbol:
            stocknamedf=ak.stock_individual_info_em(symbol=sym)
            stockname.append(stocknamedf.loc[stocknamedf['item']=='股票简称','value'][1])
            stockIPO.append(stocknamedf.loc[stocknamedf['item']=='上市时间','value'][7])
            stockIndustry.append(stocknamedf.loc[stocknamedf['item']=='行业','value'][6])
            stockTAmount.append(stocknamedf.loc[stocknamedf['item']=='总股本','value'][2])
            stockFAmount.append(stocknamedf.loc[stocknamedf['item']=='流通股','value'][3])
            stockTValue.append(stocknamedf.loc[stocknamedf['item']=='总市值','value'][4])
            stockFValue.append(stocknamedf.loc[stocknamedf['item']=='流通市值','value'][5])
            
    return {'名称':stockname,'IPO':stockIPO,'行业':stockIndustry,'总股本':stockTAmount,'流通股':stockFAmount,'总市值':stockTValue,'流通市值':stockFValue}

#画指定周期的股票烛线图
def plotDcStocks(stocksymb,periods):
    stockDict={}
    for p in periods:
        stockdf = ak.stock_zh_a_hist(symbol=stocksymb,period=p)
        stockname=getStockInfo(stocksymb)['名称']
        stockDict[p]=stockdf
        print(f'{stockname}-周期：{p}\n{stockdf}')
        plotDcStockSingle(stockdf,100)
    stockDict['name']=stockname
    return stockDict

In [ ]:
stocksymb='300339'
periods=['daily','weekly','monthly']
stockDict=plotDcStocks(stocksymb,periods)

### 4.1.2 盘口信息汇总
#### 4.1.2.1 成交和挂单明细

获取腾讯当日历史分笔数据

In [ ]:
# 根据code代码返回“交易所+代码” 信息，或者反过来
def transferExSym(code):
    if len(code)>6:
        rescode=code[-6:]
    else:
        source={}
        source['sh']=ak.stock_sh_a_spot_em()['代码'].to_list()
        source['sz']=ak.stock_sz_a_spot_em()['代码'].to_list()
        source['bj']=ak.stock_bj_a_spot_em()['代码'].to_list()
        for k in source.keys():
            if code in source[k]:
                rescode=k+code
                break
    return rescode

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#区分北交所和沪深股市，刻画市场深度。按照涨跌停板，作为极值，按照价格涨幅的measure度量区间
def priceMarketDepth(df,measure=0.01):

    open_price = df.loc[df['成交时间'] == '09:30:00', '成交价格'].iloc[0]

    # 设置涨跌停板系数
    if stocksymb == 'bj':
        limit_factor = 0.2
    else:
        limit_factor = 0.1

    # 计算最高价和最低价
    high_price = open_price * (1 + limit_factor)
    low_price = open_price * (1 - limit_factor)

    # 定义价格区间，每1%一档
    price_bins = np.round(np.arange(low_price, high_price, (high_price - low_price) * measure),2)

    # 将价格按照区间划分
    df['价格区间'] = pd.cut(df['成交价格'], bins=price_bins, right=False)

    # 按照价格区间汇总成交量
    volume_summary = df.groupby('价格区间',observed=False)['成交量'].sum().reset_index()

    # 新建 df['prop'] 列
    df['prop'] = df['性质'].apply(lambda x: -1 if x == '卖盘' else (1 if x == '买盘' else 0))
    
    # 按照价格区间汇总成交量和 prop 之和
    summary = df.groupby('价格区间',observed=False).agg({
        '成交量': 'sum',
        'prop': 'sum'
    }).reset_index()
    
    # 提取价格区间的中点和边界
    summary['价格'] = summary['价格区间'].apply(lambda x: round(x.mid,2))
    summary['起始价格'] = summary['价格区间'].apply(lambda x: round(x.left,2))
    summary['终止价格'] = summary['价格区间'].apply(lambda x: round(x.right,2))

    # 分离买盘和卖盘数据
    summary['买盘深度'] = summary['prop'].apply(lambda x: x if x > 0 else 0)
    summary['卖盘深度'] = -summary['prop'].apply(lambda x: -x if x < 0 else 0)

    # 绘制市场深度水平条形图
    fig, ax = plt.subplots(figsize=(14, 21))

    # 绘制买卖盘深度图
    ax.barh(summary['价格'], summary['买盘深度'], color='green', label='buy')
    ax.barh(summary['价格'], summary['卖盘深度'], color='red', label='sell')
    
    # 在图上标注区间价格的起始和终止价格
    for index, row in summary.iterrows():
        ax.text(row['买盘深度'] + max(summary['买盘深度'].max(), summary['卖盘深度'].max())*measure, row['价格'], f'{row["起始价格"]:.2f}', va='center', ha='left', fontsize=8, color='black')
        ax.text(-row['卖盘深度'] - max(summary['买盘深度'].max(), summary['卖盘深度'].max())*measure, row['价格'], f'{row["终止价格"]:.2f}', va='center', ha='right', fontsize=8, color='black')

    # 设置图表标题和标签
    ax.set_title('Market Depth')
    ax.set_xlabel('Depth(Amount)')
    ax.set_ylabel('Price')
    ax.legend()

    
    # 显示图表
    plt.show()
    
    return summary

In [ ]:
from rich.console import Console
from rich.table import Table

#美观显示DataFrame表格
class DataFramePretty(object):
    def __init__(self, df: pd.DataFrame, min_max_cols=[]) -> None:
        self.data = df
        self.min_max_cols = min_max_cols

    def __set_min_max_style(self, df: pd.DataFrame):
        imax = self.data.idxmax()
        imin = self.data.idxmin()

        max_tmpl = "[b on red3]{}[/b on red3]"
        min_tmpl = "[i on green3]{}[/i on green3]"

        for col in self.min_max_cols:
            col_idx = imax.index.tolist().index(col)
            val = df.iloc[imax[col], col_idx]
            df.iloc[imax[col], col_idx] = max_tmpl.format(val)

            col_idx = imin.index.tolist().index(col)
            val = df.iloc[imin[col], col_idx]
            df.iloc[imin[col], col_idx] = min_tmpl.format(val)

    def show(self):
        table = Table(
            title="DataFrame",
            title_style="i on dark_cyan",
            header_style="bold cyan",
        )

        # self.data是原始数据
        # df 是用来显示的数据
        df = self.data.copy()
        for col in df.columns:
            df[col] = df[col].astype("str")
            table.add_column(col)

        self.__set_min_max_style(df)

        for idx in range(len(df)):
            table.add_row(*df.iloc[idx].tolist())

        console = Console()
        console.print(table)

In [ ]:
#剔除中性单以后，列出活跃买卖市场深度
def plotActiveMarketDepth(stocksymbol):
    
    print(f'市场深度：{stocksymb}')
    #一次性获得历史分笔数据（当天即当天之前的交易时间；隔天即昨天）
    yesterdayTicks=ak.stock_zh_a_tick_tx_js(symbol=transferExSym(stocksymb))
    
    md=priceMarketDepth(yesterdayTicks)
    md = md[(md['买盘深度'] != 0) | (md['卖盘深度'] != 0)]
    mdp=DataFramePretty(md)
    mdp.show()
    return  md

剔除中性盘以后，主动买卖盘的市场深度

In [ ]:
plotActiveMarketDepth(stocksymb)

### 4.1.3 个股追踪
追踪股票列表

In [ ]:
focusStocks=['300576','600839','600157','000717','002607','600127','600605','600217','300727','301421','002843','600968','300159','601688']
getInfo=getStockInfo(focusStocks)
Analist=pd.DataFrame({'代码':focusStocks,'名称':getInfo['名称'],'IPO':getInfo['IPO'],'行业':getInfo['行业'],'流通股':getInfo['流通股'],'总股本':getInfo['总股本']})
Analist['流通股']=Analist['流通股'].astype(int)
Analist['总股本']=Analist['总股本'].astype(int)
Analist['流通率']=Analist['流通股']/Analist['总股本']
Analist

时序数据库准备

In [ ]:
import dolphindb as ddb
from concurrent.futures import ThreadPoolExecutor
import dolphindb.settings as keys

s = ddb.session("localhost", 8848, "admin", "123456")
s.run("quote_tab = table(1:0, `timestamp`sym`sell_5`sell_5_vol`sell_4`sell_4_vol`sell_3`sell_3_vol`sell_2`sell_2_vol`sell_1`sell_1_vol`buy_1`buy_1_vol`buy_2`buy_2_vol`buy_3`buy_3_vol`buy_4`buy_4_vol`buy_5`buy_5_vol`最新`均价`涨幅`涨跌`总手`金额`换手`量比`最高`最低`今开`昨收`涨停`跌停`外盘`内盘, [TIMESTAMP, SYMBOL, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE,DOUBLE,DOUBLE])")
quote_tab = s.table(data="quote_tab")
dbPath="dfs://quotes"

if not s.existsDatabase(dbPath):
    db = s.database(dbPath=dbPath, partitionType=keys.LIST, partitions=focusStocks,engine='TSDB')
    db.createTable(quote_tab, "quotes", ["sym"])
    
qt = s.table(dbPath=dbPath, data="quotes")


In [ ]:
stockCharts,marketDepths={},{}

for f in focusStocks:
    print('='*75)
    stockCharts[f]=plotDcStocks(f,periods)
    stockname=stockCharts[f]['name']
    print(f'{stockname}（{f}）的市场深度：')
    marketDepths[f]=plotActiveMarketDepth(f)
    

In [ ]:
stock_bid_ask_em_df = ak.stock_bid_ask_em(symbol="600839")
stock_bid_ask_em_df

In [ ]:
import akshare as ak
import dolphindb as ddb
import pandas as pd
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# 初始化 DolphinDB 连接
s = ddb.session()
s.connect("localhost", 8848, "admin", "123456")  # 替换为实际的主机、端口、用户名和密码

# 创建 DolphinDB 表
s.run("""
if (existsDatabase("dfs://Quote")) {
    dropDatabase("dfs://Quote")
}
t = table(1:0, `timestamp`sym`sell_5`sell_5_vol`sell_4`sell_4_vol`sell_3`sell_3_vol`sell_2`sell_2_vol`sell_1`sell_1_vol`buy_1`buy_1_vol`buy_2`buy_2_vol`buy_3`buy_3_vol`buy_4`buy_4_vol`buy_5`buy_5_vol`最新`均价`涨幅`涨跌`总手`金额`换手`量比`最高`最低`今开`昨收`涨停`跌停`外盘`内盘, [TIMESTAMP, SYMBOL, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE])
database("dfs://Quote", VALUE, t)
""")

def is_within_trading_hours(current_time):
    # 定义交易时间段
    morning_start = current_time.replace(hour=9, minute=15, second=0, microsecond=0)
    morning_end = current_time.replace(hour=11, minute=30, second=0, microsecond=0)
    afternoon_start = current_time.replace(hour=13, minute=0, second=0, microsecond=0)
    afternoon_end = current_time.replace(hour=15, minute=0, microsecond=0)
    
    return (morning_start <= current_time <= morning_end) or (afternoon_start <= current_time <= afternoon_end)

def fetch_and_store(symbol):
    # 获取当前时间
    current_time = datetime.now()

    if is_within_trading_hours(current_time):fgggttryy
        # 获取股票数据
        df = ak.stock_bid_ask_em(symbol=symbol)

        # 转换为字典
        data_dict = df.set_index('item').to_dict()['value']

        # 添加时间戳和股票代码到数据字典
        data_dict['timestamp'] = current_time
        data_dict['sym'] = symbol

        # 转换数据为 DataFrame
        transformed_df = pd.DataFrame([data_dict])

        # 将数据插入到 DolphinDB
        s.upload({"data": transformed_df})
        s.run("""
        append!(loadTable("dfs://Quote", "t"), data)
        """)]]
        ]

while True:
    
    # 使用多线程获取多个股票代码的数据
    with ThreadPoolExecutor(max_workers=len(focusStocks)) as executor:
        executor.map(fetch_and_store, focusStocks)

    # 每3秒检查一次
    time.sleep(3)
